In [ ]:
# Analysis: Bakayoko replacement
# Name: Yasin Tunçbilek
# Date: 9 February 2024

In [1]:
# Library to send HTTP requests
import requests

In [7]:
# URL of Eredivisie statistics
eredivisie_url = "https://fbref.com/en/comps/23/stats/Eredivisie-Stats"

In [8]:
# Download HTML of page
data = requests.get(eredivisie_url)

In [10]:
# View HTML
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202402061" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://cmp.quantcast.com\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \n\t\t    \'/choice.js?tag_version=V2\');\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insertBef

In [12]:
# Library to parse HTML
from bs4 import BeautifulSoup

In [38]:
# Initialise library with downloaded HTML
soup = BeautifulSoup(data.text)

In [52]:
# Select menu where URLs to various statistics reside in
menu = soup.select('li.full.current.hasmore')[0]

In [54]:
# Find all tags where URLs to various statistics reside in
statistics_urls = menu.find_all('a')

In [55]:
statistics_urls

[<a href="/en/comps/23/stats/Eredivisie-Stats">Standard Stats</a>,
 <a href="/en/comps/23/keepers/Eredivisie-Stats">Goalkeeping</a>,
 <a href="/en/comps/23/keepersadv/Eredivisie-Stats">Advanced Goalkeeping</a>,
 <a href="/en/comps/23/shooting/Eredivisie-Stats">Shooting</a>,
 <a href="/en/comps/23/passing/Eredivisie-Stats">Passing</a>,
 <a href="/en/comps/23/passing_types/Eredivisie-Stats">Pass Types</a>,
 <a href="/en/comps/23/gca/Eredivisie-Stats">Goal and Shot Creation</a>,
 <a href="/en/comps/23/defense/Eredivisie-Stats">Defensive Actions</a>,
 <a href="/en/comps/23/possession/Eredivisie-Stats">Possession</a>,
 <a href="/en/comps/23/playingtime/Eredivisie-Stats">Playing Time</a>,
 <a href="/en/comps/23/misc/Eredivisie-Stats">Miscellaneous Stats</a>]

In [28]:
# Select menu where URLs to various statistics reside
links = soup.find_all('a')

In [29]:
links = [l.get("href") for l in links]

In [35]:
links = [l for l in links if l and '/en/comps/23/' in l]

In [36]:
links

['https://fbref.com/en/comps/23/stats/Eredivisie-Stats',
 'https://fbref.com/en/comps/23/stats/Eredivisie-Stats',
 '/en/comps/23/2022-2023/stats/2022-2023-Eredivisie-Stats',
 '/en/comps/23/Eredivisie-Stats#all_league_structure',
 '/en/comps/23/history/Eredivisie-Seasons',
 '/en/comps/23/Eredivisie-Stats',
 '/en/comps/23/schedule/Eredivisie-Scores-and-Fixtures',
 '/en/comps/23/nations/Eredivisie-Nationalities',
 '/en/comps/23/stats/Eredivisie-Stats',
 '/en/comps/23/keepers/Eredivisie-Stats',
 '/en/comps/23/keepersadv/Eredivisie-Stats',
 '/en/comps/23/shooting/Eredivisie-Stats',
 '/en/comps/23/passing/Eredivisie-Stats',
 '/en/comps/23/passing_types/Eredivisie-Stats',
 '/en/comps/23/gca/Eredivisie-Stats',
 '/en/comps/23/defense/Eredivisie-Stats',
 '/en/comps/23/possession/Eredivisie-Stats',
 '/en/comps/23/playingtime/Eredivisie-Stats',
 '/en/comps/23/misc/Eredivisie-Stats',
 '/en/comps/23/Eredivisie-Stats',
 '/en/comps/23/schedule/Eredivisie-Scores-and-Fixtures',
 '/en/comps/23/stats/Ered